# load tons of datasets (~60,000 RNAseq samples)

In [2]:
from taigapy import TaigaClient
tc = TaigaClient()

from depmapomics import tracker as track
from depmapomics import expressions

from genepy.utils import helper as h

import dalmatian as dm
from gsheets import Sheets
import pandas as pd

from anndata import AnnData, read_h5ad

MY_ID = '~/.client_secret.json'
MYSTORAGE_ID = "~/.storage.json"

Sheets.from_files(MY_ID, MYSTORAGE_ID)
#autoreload
%load_ext autoreload
%autoreload 2

## CCLE + TCGA

In [2]:
# load from taiga public (figshare link)
# load internal expression,
# latest version can be found at https://depmap.org/portal/download/
# can also be loaded like so pd.read_csv('gs://ccle_default_params/celligner_ex/CCLE_expression.csv.gz', index_col=0)
CCLE_expression = tc.get(name='internal-21q3-fe4c',
                         file='CCLE_expression_full')  #40,000x1,500


# load  TCGA expression
# this dataset was generated from  ,using this script: 
# caan be found here: pd.read_csv('gs://ccle_default_params/celligner_ex/TCGA_expression.csv.gz', index_col=0)
TCGA_expression = tc.get(name='celligner-input-9827',
                       file='tumor_expression') # 40,000x13,000

No dataset version provided. Using version 16.
No dataset version provided. Using version 1.


In [4]:
# loading annotations
#CCLE_annotation = track.getTracker() # the function uses pygsheets to load this: REFSHEET_URL=https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY
# Sheets.from_files(MY_ID, MYSTORAGE_ID).get(REFSHEET_URL).sheets[0].to_frame(index_col=0)
# you can also get it from pd.read_csv('gs://ccle_default_params/celligner_ex/CCLE_annotation.csv.gz', index_col=0)


# can be loaded from 
# pd.read_csv('gs://ccle_default_params/celligner_ex/TCGA_annotation.csv.gz', index_col=0)
TCGA_annotation = tc.get(name='celligner-input-9827',
                         file='tumor_annotations') # generated manually 

No dataset version provided. Using version 1.


In [8]:
set(pd.read_csv('gs://celligner/clinical_TumorCompendium_v11_PolyA_2020-04-09.tsv.gz', sep='\t')['disease'])

{'Ewing sarcoma',
 'INI-deficient soft tissue sarcoma NOS',
 'NUT midline carcinoma',
 'PEComa',
 'Sertoli-Leydig cell tumor, retiform',
 'acinar cell carcinoma',
 'acute leukemia',
 'acute leukemia of ambiguous lineage',
 'acute lymphoblastic leukemia',
 'acute megakaryoblastic leukemia',
 'acute myeloid leukemia',
 'acute undifferentiated leukemia',
 'adrenocortical adenoma',
 'adrenocortical cancer',
 'adrenocortical carcinoma',
 'alveolar rhabdomyosarcoma',
 'alveolar soft part sarcoma',
 'angiosarcoma',
 'atypical teratoid/rhabdoid tumor',
 'bladder urothelial carcinoma',
 'breast invasive carcinoma',
 'cervical & endocervical cancer',
 'cholangiocarcinoma',
 'choroid plexus carcinoma',
 'chronic myelogenous leukemia (S02), acute lymphoblastic leukemia (S01)',
 'clear cell carcinoma of cervix',
 'colon adenocarcinoma',
 'craniopharyngioma',
 'dedifferentiated liposarcoma',
 'desmoplastic small round cell tumor',
 'diffuse large B-cell lymphoma',
 'dysembryoplastic neuroepithelial 

In [5]:
TCGA_annotation

,sampleID,age_at_dx,pedaya,gender,site_id,site_donor_id,site_sampleid,lineage,disease,TCGA_disease,TCGA_primary_site,sample_type,CPE,ABSOLUTE,Cancer type,subtype,UMAP_1,UMAP_2,cluster
0,TH27_1241_S01,NaN,Unknown,unknown,TH27,None,None,central_nervous_system,Brain Cancer,None,None,None,NaN,NaN,None,glioma,5.433872,10.000675,52
1,TARGET-40-0A4I65-01A-01R,17.86,"Yes, age < 30 years",male,TARGET,TARGET-40-0A4I65,SRR1811136,bone,Bone Cancer,None,None,None,NaN,NaN,None,osteosarcoma,7.273973,-0.284139,35
2,THR24_1965_S01,13.27,"Yes, age < 30 years",male,THR24,SJHGG060,SJHGG060_A,central_nervous_system,Brain Cancer,None,None,None,NaN,NaN,None,glioma,4.895308,13.367768,27
3,THR24_2080_S01,3.75,"Yes, age < 30 years",male,THR24,SJMEL001005,SJMEL001005_D1,skin,Skin Cancer,None,None,None,NaN,NaN,None,None,6.300287,5.229102,4
4,THR20_0494_S01,6.00,"Yes, age < 30 years",female,THR20,icgc/_EGAR00001049890_RNA_tumor_ICGC_MB46,EGAR00001049890_RNA_tumor_ICGC_MB46,central_nervous_system,Brain Cancer,None,None,None,NaN,NaN,None,medulloblastoma,5.415814,9.806541,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12231,TCGA-95-7947-01,67.00,No,male,TCGA,TCGA-95-7947,TCGA-95-7947-01,lung,Lung Cancer,lung adenocarcinoma,Lung,Primary Tumor,0.7556,0.66,LUAD,lung adenocarcinoma,-5.478718,-3.438399,0
12232,TCGA-VQ-AA6F-01,57.00,No,male,TCGA,TCGA-VQ-AA6F,TCGA-VQ-AA6F-01,gastric,Gastric Cancer,stomach adenocarcinoma,Stomach,Primary Tumor,NaN,NaN,None,stomach adenocarcinoma,-7.366354,0.347824,1
12233,TCGA-BR-8588-01,55.00,No,female,TCGA,TCGA-BR-8588,TCGA-BR-8588-01,gastric,Gastric Cancer,stomach adenocarcinoma,Stomach,Primary Tumor,NaN,NaN,None,stomach adenocarcinoma,-7.727499,0.404288,1
12234,TCGA-24-2254-01,66.00,No,female,TCGA,TCGA-24-2254,TCGA-24-2254-01,ovary,Ovarian Cancer,ovarian serous cystadenocarcinoma,Ovary,Primary Tumor,0.9309,0.81,OV,ovarian serous cystadenocarcinoma,0.769441,3.263780,10


In [ ]:
TCGA_annotation

In [4]:
CCLE_annotation.iloc[0, :-25]

arxspan_id                                                ACH-001188
version                                                            1
sm_id                                                            NaN
PDO                                                              NaN
datatype                                                         wgs
                                         ...                        
dsmzlink                                                     ACC-209
sequencing_date                                            13-9-2020
release_date                                                     NaN
bam_qc             ['gs://fc-secure-fd4a7e9f-cb15-423c-9805-04995...
processing_qc      ['gs://fc-secure-fd4a7e9f-cb15-423c-9805-04995...
Name: CDS-rIh3Gy, Length: 63, dtype: object

In [5]:
len(CCLE_annotation[(CCLE_annotation.blacklist==0)&(CCLE_annotation.version>1)&(CCLE_annotation.datatype=="rna")]) 
# we have .. replicates in CCLE

79

In [6]:
rename = {
'Acquisition Site': "collection_site",
'Actual Seq Technology': 'sequencer',
'Age At Acquisition (Years)': "age",
'age_at_dx':"age", 
'Age': "age",
'Aggregated': "aggregated",
'Assay title': 'assay',
'arxspan_id': "tissue_id",
'biopsy_tissue': "collection_site",
'Case ID': 'participant_id', 
'CCLF_ID': 'sample_id',
'Clinical Tumor Diagnosis': 'disease_type',
'Collaborator Participant ID': "participant_id",
'collection': 'collection_site', 
'Contamination % (First Agg)': 'contamination', 
'Contamination %': "contamination",
'Culture Medium': "media_type",
'Days to First Agg': 'exp_date', 
'Diagnosis': 'disease_subtype',
'disease': "disease_type", 
'Expansion Status': "expansion",
'External ID for BAM': "tissue_id",
'gender': "sex",
'Gender': "sex",
'Inferred_Ethnicity': "ethnicity",
'lineage': 'tissue_type',
'mediatype':"media_type",
'Original Material Type': 'history',
'Passage Number': "passage_number",
'Phase':'stage',
'Primary Disease': "disease_type",
'Primary Site': 'collection_site',
'primary_disease': "disease_type", 
'Product': 'preparation',
'Race': "ethnicity",
'RNA_Seq_cancertype': 'disease_type', 
'sample_source': 'participant_id',
'Sample_type': 'lineage', 
'sampleID': 'sample_id',
'Sample_id': 'sample_id',
'Sequencing on Tissue or Cell model? (MT confirm)': 'cell_type', 
'Sex': "sex",
'tissue': "lineage",
'Participant ID': 'participant_id',
'RIN score from PAXgene tissue Aliquot': 'rin_score',
'RIN': 'rin_score',
'Cell types level 3': 'tissue_type',
'Age_bin':'age',
'cohort': "disease_type",
'site_donor_id': "participant_id",
'site_id': "tissue_id",
'Site.of.Specimen ': "collection_site",
'stripped_cell_line_name': "sample_id", 
'subtype': 'disease_subtype', 
'tc': 'tumor_purity',
'Therapy': "therapy",
'Tissue Site': "collection_site",
'Tissue Status': "metastatic/primary",
'Tumor Percent': "contamination",
'Tumor Type': "metastatic/primary",
'Tumor_type': 'cell_type', 
'type': "cell_type",
'Type': "cell_type",
'Sample Type': "cell_type",
'sample_type': 'sample_type',
'dataset': 'dataset',
'ends': 'ends',
'reference': 'reference',
'method': 'method',
}

In [7]:
CCLE_annotation = CCLE_annotation.rename(columns=rename)
CCLE_annotation = CCLE_annotation[(set(['origin', 'sequencing_type', 'doublingt','hasebv']) | set(rename.values())) & set(CCLE_annotation.columns)]
CCLE_annotation['method']="bulk"
CCLE_annotation['cell_type']="historical_CL; 2D"
CCLE_annotation['sequencer']="Illumina Hiseq 2000"
CCLE_annotation['ends']="paired end"
CCLE_annotation['reference']="hg38"
CCLE_annotation['dataset']="ccle"

In [8]:
TCGA_annotation.iloc[0]

sampleID                      TH27_1241_S01
age_at_dx                               NaN
pedaya                              Unknown
gender                              unknown
site_id                                TH27
site_donor_id                          None
site_sampleid                          None
lineage              central_nervous_system
disease                        Brain Cancer
TCGA_disease                           None
TCGA_primary_site                      None
sample_type                            None
CPE                                     NaN
ABSOLUTE                                NaN
Cancer type                            None
subtype                              glioma
UMAP_1                             5.433872
UMAP_2                            10.000675
cluster                                  52
Name: 0, dtype: object

In [9]:
TCGA_annotation = TCGA_annotation.rename(columns=rename)
TCGA_annotation = TCGA_annotation[set(rename.values()) & set(TCGA_annotation.columns)]
TCGA_annotation['method']="bulk"
TCGA_annotation['cell_type']="tumor"
TCGA_annotation['metastasis']="Primary"
TCGA_annotation['sequencer']= "Illumina Hiseq 2000"
TCGA_annotation['reference']= "hg38"
TCGA_annotation['ends']="paired end"
TCGA_annotation['dataset']="tcga"

TCGA_annotation = TCGA_annotation.set_index('sample_id')

In [10]:
CCLE_annotation = CCLE_annotation[set(CCLE_annotation.columns) & set(rename.values())]

In [1]:
set(CCLE_expression.index) & set(TCGA_expression.index)

NameError: name 'CCLE_expression' is not defined

In [3]:
CCLE_expression.columns = [i.split('(')[-1][:-1] if '(' in i else i for i in CCLE_expression.columns]

NameError: name 'CCLE_expression' is not defined

In [46]:
len(set(CCLE_expression.columns) - set(TCGA_expression.columns)), len(set(CCLE_expression.columns) & set(TCGA_expression.columns)), len(set(TCGA_expression.columns) - set(CCLE_expression.columns)),

(22411, 29643, 4390)

## CCLF

In [69]:
cclf_orga_info = tc.get(name='cclf-organoids-c23d', version=1, file='cclf_orga_info')
cclf_orga_info.iloc[0]

Genomic_Seq                          Pending
RNA_Seq_cancertype              BREAST_BASAL
RNA_Seq_marker                          None
CCLF_ID                       CCLF_cRCRF1048
Diagnosis             Advanced breast cancer
Subtype_patient                ER/PR+, HER2-
Tumor_type                        Metastatic
Sample_type                 Pleural effusion
Name: 0, dtype: object

In [71]:
cclf_orga_info = cclf_orga_info.rename(columns=rename)
cclf_orga_info = cclf_orga_info[set(rename.values()) & set(cclf_orga_info.columns)]
cclf_orga_rnaseq = tc.get(name='cclf-organoids-c23d', version=1, file='cclf_orga_rnaseq').T # 40,000x24
cclf_orga_info.index = [i.split("_")[1] for i in cclf_orga_info.sample_id]
cclf_orga_rnaseq.index = [i.split('_')[0][:-1] for i in cclf_orga_rnaseq.index]

In [72]:
cclf_orga_info['sequencer'] = "illumina Hiseq 2000"
cclf_orga_info['method'] = "bulk"
cclf_orga_info['cell_type'] = "organoid; cancer"
cclf_orga_info['reference']= "hg38"
cclf_orga_info['ends']="paired end"
cclf_orga_info['dataset'] = "cclf_orga"

In [73]:
ann = pd.concat([ann, cclf_orga_info])
ann

,participant_id,disease_type,tissue_type,age,disease_subtype,sex,tissue_id,method,cell_type,metastasis,...,reference,ends,dataset,media_type,sequencing_type,collection_site,sample_id,hasebv,doublingt,lineage
TH27_1241_S01,None,Brain Cancer,central_nervous_system,NaN,glioma,unknown,TH27,bulk,tumor,Primary,...,hg38,paired end,tcga,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TARGET-40-0A4I65-01A-01R,TARGET-40-0A4I65,Bone Cancer,bone,17.86,osteosarcoma,male,TARGET,bulk,tumor,Primary,...,hg38,paired end,tcga,NaN,NaN,NaN,NaN,NaN,NaN,NaN
THR24_1965_S01,SJHGG060,Brain Cancer,central_nervous_system,13.27,glioma,male,THR24,bulk,tumor,Primary,...,hg38,paired end,tcga,NaN,NaN,NaN,NaN,NaN,NaN,NaN
THR24_2080_S01,SJMEL001005,Skin Cancer,skin,3.75,None,male,THR24,bulk,tumor,Primary,...,hg38,paired end,tcga,NaN,NaN,NaN,NaN,NaN,NaN,NaN
THR20_0494_S01,icgc/_EGAR00001049890_RNA_tumor_ICGC_MB46,Brain Cancer,central_nervous_system,6.0,medulloblastoma,female,THR20,bulk,tumor,Primary,...,hg38,paired end,tcga,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cRCRF1058,NaN,FIBROBLAST,NaN,NaN,Ovarian cancer,NaN,NaN,bulk,organoid; cancer,NaN,...,hg38,paired end,cclf_orga,NaN,NaN,NaN,CCLF_cRCRF1058,NaN,NaN,Ascites
cRCRF1004,NaN,OVARY,NaN,NaN,Ovarian serious cystadenocarcinoma,NaN,NaN,bulk,organoid; cancer,NaN,...,hg38,paired end,cclf_orga,NaN,NaN,NaN,CCLF_cRCRF1004,NaN,NaN,Pleural effusion
cRCRF1040,NaN,None,NaN,NaN,Advanced breast cancer,NaN,NaN,bulk,organoid; cancer,NaN,...,hg38,paired end,cclf_orga,NaN,NaN,NaN,CCLF_cRCRF1040,NaN,NaN,Pleural effusion
KL1220,NaN,None,NaN,NaN,Breast met to brain,NaN,NaN,bulk,organoid; cancer,NaN,...,hg38,paired end,cclf_orga,NaN,NaN,NaN,CCLF_KL1220,NaN,NaN,Brain met resection


In [80]:
cclf_orga_rnaseq.columns = [i.split('.')[0] for i in cclf_orga_rnaseq.columns]

In [81]:
len(set(CCLE_expression.columns) - set(cclf_orga_rnaseq.columns)), len(set(CCLE_expression.columns) & set(cclf_orga_rnaseq.columns)), len(set(cclf_orga_rnaseq.columns) - set(CCLE_expression.columns)),

(12002, 40052, 560)

In [121]:
#cclf other
cclfrna = dm.WorkspaceManager("nci-mimoun-bi-org/CCLF_RNA_2_0").get_samples() #40,000x160

cclfrna_anno = cclfrna[["external_id_rna"]].replace({'NA': np.nan})

cclfrna_annot = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/1O9IV_v2vMbebkk_KDWu3LdKBQ16c8lThJKiiWvRxMUo").sheets[2].to_frame()

cclfrna_annot2 = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/1O9IV_v2vMbebkk_KDWu3LdKBQ16c8lThJKiiWvRxMUo").sheets[3].to_frame()
# get it  from https://docs.google.com/spreadsheets/d/1O9IV_v2vMbebkk_KDWu3LdKBQ16c8lThJKiiWvRxMUo and from Terra: in the workspace: nci-mimoun-bi-org/CCLF_RNA_2_0

In [140]:
#files, failed, _, _, lowqual, _ = await expressions.postProcess("nci-mimoun-bi-org/CCLF_RNA_2_0", "all_samples", samplesetToLoad = "all_samples", compute_enrichment=False, trancriptLevelCols = ['rsem_transcripts_expected_count', 'rsem_transcripts_tpm'], geneLevelCols = ["rsem_genes_tpm", "rsem_genes_expected_count"], save_output="data/")
#cclfrna = files['rsem_genes_tpm']
cclfrna = pd.read_csv('data/expression_genes_tpm.csv.gz', index_col=0)
cclfrna = np.log2(1+cclfrna)

In [122]:
ina = (cclfrna_annot2['Passage Number'].isna() | (cclfrna_annot2['Passage Number']=="Unknown")) & ~(cclfrna_annot2["Passage Number on Receipt"].isna() | (cclfrna_annot2["Passage Number on Receipt"]=="Unknown"))
cclfrna_annot2.loc[ina, "Passage Number"] = cclfrna_annot2.loc[ina, "Passage Number on Receipt"].values

ina = (cclfrna_annot2['Gender'].isna() | (cclfrna_annot2['Gender']=="Unknown")) & ~(cclfrna_annot2["Gender.1"].isna() | (cclfrna_annot2["Gender.1"]=="Unknown"))
cclfrna_annot2.loc[ina, "genderA"] = cclfrna_annot2.loc[ina, "Gender.1"].values

ina = (cclfrna_annot2['Gender'].isna() | (cclfrna_annot2['Gender']=="Unknown")) & ~(cclfrna_annot2["FP Gender"].isna() | (cclfrna_annot2["FP Gender"]=="Unknown"))
cclfrna_annot2.loc[ina, "Gender"] = cclfrna_annot2.loc[ina, "FP Gender"].values

ina = (cclfrna_annot2['Race'].isna() | (cclfrna_annot2['Race']=="Unknown")) & ~(cclfrna_annot2["Ethnicity"].isna() | (cclfrna_annot2["Ethnicity"]=="Unknown"))
cclfrna_annot2.loc[ina, "Race"] = cclfrna_annot2.loc[ina, "Ethnicity"].values


In [123]:
cclfrna_annot2.iloc[0]

Sample ID                                                               SM-5KIB3
Collaborator Participant ID                                               3T3/J2
Collaborator Sample ID                                                 3T3/J2_CL
Sample LSID                             broadinstitute.org:bsp.prod.sample:5KIB3
Sample Status                                                          INSTORAGE
                                                     ...                        
Volume Changed                 Volume changed from 100.0ul to 88.0ul, Volume ...
XL20 Used for Total Weight                                                   NaN
XL20 Used to Tare                                                            NaN
rRNA Height Ratio 28S/18S                                                    NaN
genderA                                                                      NaN
Name: 0, Length: 126, dtype: object

In [124]:
cclfrna_annot.iloc[0]

Data File Type? (CCLF QC, CCLF Full Model Characterization, Other R&D, Other Exclude)                      CCLF Full Model Characterization
Collaborator Sample ID                                                                                                            AB002T_BT
Related project catagory                                                                                                           CCLF-mix
Sequencing purpose? (Cell model genomic profiling/Driver mutation discovery/Mouse material)(MT confirm)        Cell model genomic profiling
Sequencing on Tissue or Cell model? (MT confirm)                                                                                 Cell model
                                                                                                                         ...               
Mean Coverage (Raw)                                                                                                                     0.0
Median Coverage (Raw

In [125]:
cclfrna_annot2 = cclfrna_annot2.set_index('Collaborator Sample ID')[["Age",
"Gender",
"Tumor Type",
"Tissue Site",
"Primary Disease",
"Race",
"Culture Medium",
"Passage Number",]]

In [126]:
cclfrna_annot = cclfrna_annot[[
 'Sequencing on Tissue or Cell model? (MT confirm)',
 'External ID for BAM',
 'Product',
 'RIN',
 'Collaborator Sample ID',
 'Original Material Type',
 'Collaborator Participant ID',
 'Aggregated',
 'Actual Seq Technology',
 'Contamination %',
]].set_index('Collaborator Sample ID', drop=True)

In [127]:
for val in h.dups(cclfrna_annot2.index):
    for i in range(len(cclfrna_annot2.loc[val])-1):
        if cclfrna_annot2.loc[val].iloc[0].isna().sum() > cclfrna_annot2.loc[val].iloc[i+1].isna().sum():
            cclfrna_annot2.iloc[np.argwhere(cclfrna_annot2.index == val).flatten()[0]] = cclfrna_annot2.loc[val].iloc[i+1].values
cclfrna_annot2 = cclfrna_annot2[~cclfrna_annot2.index.duplicated(keep='first')]

In [128]:
for val in h.dups(cclfrna_annot.index):
    for i in range(len(cclfrna_annot.loc[val])-1):
        if cclfrna_annot.loc[val].iloc[0].isna().sum() > cclfrna_annot.loc[val].iloc[i+1].isna().sum():
            cclfrna_annot.iloc[np.argwhere(cclfrna_annot.index == val).flatten()[0]] = cclfrna_annot.loc[val].iloc[i+1].values
cclfrna_annot = cclfrna_annot[~cclfrna_annot.index.duplicated(keep='first')]

In [129]:
cclfrna_annot = pd.concat([cclfrna_annot, cclfrna_annot2], axis=1)

In [130]:
for i, val in cclfrna_annot.iterrows():
    cclfrna_anno.loc[cclfrna_anno.external_id_rna==i, cclfrna_annot.columns] = val.values
del cclfrna_annot

In [132]:
cclfrna_anno['reference']= "hg38"
cclfrna_anno['ends']="paired end"
cclfrna_anno['method'] = "bulk"
cclfrna_anno['dataset'] = "cclf"

cclfrna_anno = cclfrna_anno.rename(columns=rename)

In [136]:
ann = pd.concat([cclfrna_anno, CCLE_annotation])
ann

,external_id_rna,cell_type,tissue_id,preparation,rin_score,history,participant_id,aggregated,sequencer,contamination,...,reference,ends,method,dataset,tissue_type,sequencing_type,disease_subtype,sample_id,hasebv,doublingt
AA02-Tumor-SM-5LB98,AA02T,Cell model,AA02T,Tru-Seq Strand Specific Large Insert RNA Seque...,NaN,"Cells:Cell Line, Viable",AA02,0.0,HISEQ_2500,1.91,...,hg38,paired end,bulk,cclf,NaN,NaN,NaN,NaN,NaN,NaN
AA13-Tumor-SM-GXH5W,AA13T_Repeat,Cell model,AA13T_Repeat,Tru-Seq Strand Specific Large Insert RNA Seque...,NaN,Cells:Pellet frozen,AA13,0.0,HISEQ_2500,0.18,...,hg38,paired end,bulk,cclf,NaN,NaN,NaN,NaN,NaN,NaN
AA22-Tumor-SM-ALFLY,AA22T,Cell model,AA22T,Tru-Seq Strand Specific Large Insert RNA Seque...,NaN,Cells:Growing,AA22,0.0,HISEQ_2500,0.60,...,hg38,paired end,bulk,cclf,NaN,NaN,NaN,NaN,NaN,NaN
AA22-Tumor-SM-GXH2L,AA22T_OPAC,Cell model,AA22T_OPAC,Tru-Seq Strand Specific Large Insert RNA Seque...,NaN,Cells:Pellet frozen,AA22,0.0,HISEQ_2500,0.42,...,hg38,paired end,bulk,cclf,NaN,NaN,NaN,NaN,NaN,NaN
AA26-Tumor-SM-AU5YZ,AA26T,Cell model,AA26T,Tru-Seq Strand Specific Large Insert RNA Seque...,NaN,Cells:Pellet frozen,AA26,0.0,HISEQ_2500,0.53,...,hg38,paired end,bulk,cclf,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CDS-qbHB8J,NaN,historical_CL; 2D,ACH-002498,NaN,NaN,NaN,PT-kiG6X0x0,NaN,Illumina Hiseq 2000,NaN,...,hg38,paired end,bulk,ccle,lymphocyte,NaN,non_hodgkin_lymphoma,SUDHL2,False,NaN
CDS-y7yI24,NaN,historical_CL; 2D,ACH-001704,NaN,NaN,NaN,PT-9Sjq1s2c,NaN,Illumina Hiseq 2000,NaN,...,hg38,paired end,bulk,ccle,skin,NaN,melanoma,VMRCMELG,False,60.0
CDS-cteUkO,NaN,historical_CL; 2D,ACH-001709,NaN,NaN,NaN,PT-55Pt5qaS,NaN,Illumina Hiseq 2000,NaN,...,hg38,paired end,bulk,ccle,lymphocyte,NaN,lymphoma_unspecified,WSUNHL,False,57.0
CDS-dQKiht,NaN,historical_CL; 2D,ACH-003020,NaN,NaN,NaN,PT-TRbh2Ida,NaN,Illumina Hiseq 2000,NaN,...,hg38,paired end,bulk,ccle,breast,NaN,normal,HCC1143_NORMAL,NaN,NaN


In [138]:
set(CCLE_expression.index) & set(cclfrna.index)

set()

In [137]:
cclfrna.columns = [i.split('(')[-1][:-1] if '(' in i else i for i in cclfrna.columns]

In [139]:
len(set(CCLE_expression.columns) - set(cclfrna.columns)), len(set(CCLE_expression.columns) & set(cclfrna.columns)), len(set(cclfrna.columns) - set(CCLE_expression.columns)),

(6843, 45211, 10)

## MET500 and PDXs

In [33]:
set(Novartis_PDX_ann['lineage'].astype(str) + "_" + Novartis_PDX_ann['subtype'].astype(str))

{'NS_adenocarcinoma',
 'NS_neuroendocrine',
 'autonomic_ganglia_NS',
 'bile_duct_adenocarcinoma',
 'bile_duct_adenosquamous_carcinoma',
 'bone_NS',
 'bone_chondroblastic',
 'bone_dedifferentiated',
 'bone_myxoid',
 'breast_NS',
 'breast_ductal_carcinoma',
 'breast_ductal_carcinoma_in_situ',
 'breast_lobular_carcinoma',
 'breast_metaplastic_carcinoma',
 'central_nervous_system_NS',
 'central_nervous_system_astrocytoma_Grade_IV',
 'central_nervous_system_ependymoma_Grade_III-IV',
 'colorectal_NS',
 'colorectal_adenocarcinoma',
 'colorectal_squamous_cell_carcinoma',
 'esophagus_adenocarcinoma',
 'esophagus_squamous_cell_carcinoma',
 'gastric_adenocarcinoma',
 'gastric_diffuse_adenocarcinoma',
 'kidney_NS',
 'kidney_clear_cell_renal_cell_carcinoma',
 'kidney_transitional_cell_carcinoma',
 'liver_hepatocellular_carcinoma',
 'lung_NS',
 'lung_adenocarcinoma',
 'lung_basaloid_carcinoma',
 'lung_large_cell_carcinoma',
 'lung_mixed_adenosquamous_carcinoma',
 'lung_non_small_cell_carcinoma',
 'l

In [25]:
set(pediatric_PDX_ann["lineage"].astype(str) + "_" + pediatric_PDX_ann["subtype"].astype(str) + "_" + pediatric_PDX_ann["Histology.Detailed"].astype(str) + "_" + pediatric_PDX_ann["Histology-Detailed2"].astype(str) + "_" + pediatric_PDX_ann["Molecular-Subtype-Brain"].astype(str))

{'blood_ALL_BCP-ALL_BCP-ALL_None',
 'blood_ALL_ETP-ALL_ETP-ALL_None',
 'blood_ALL_MLL-ALL_MLL-ALL_None',
 'blood_ALL_Ph+-ALL_Ph+-ALL_None',
 'blood_ALL_Ph-likeALL_Ph-likeALL_None',
 'blood_ALL_T-ALL_T-ALL_None',
 'bone_Ewing Sarcoma_Ewing Sarcoma_Ewing Sarcoma_None',
 'bone_Extracranial Rhabdoid_Extracranial Rhabdoid_Extracranial Rhabdoid_None',
 'bone_Osteosarcoma_Osteosarcoma_Osteosarcoma_None',
 'central_nervous_system_CNS germinoma_CNS germinoma_CNS germinoma_None',
 'central_nervous_system_DIPG_DIPG_DIPG_IDH/H3 WT',
 'central_nervous_system_ETMR_ETMR_ETMR_ETMR',
 'central_nervous_system_Ependymoblastoma_Ependymoblastoma_Ependymoblastoma_Ependymoblastoma',
 'central_nervous_system_Ependymoma_Ependymoma_PF-EPN-A_PF-EPN-A',
 'central_nervous_system_Ependymoma_Ependymoma_PF-EPN-B_PF-EPN-B',
 'central_nervous_system_Ependymoma_Ependymoma_ST-EPN-RELA_ST-EPN-RELA',
 'central_nervous_system_Ependymoma_Ependymoma_ST-EPN-YAP1_ST-EPN-YAP1',
 'central_nervous_system_Glioblastoma_Glioblastoma_

In [30]:
set( met500_meta['tissue'].astype(str)+"_" +met500_meta['cohort'].astype(str))

{'None_HNSC',
 'None_MISC',
 'None_SECR',
 'adrenal_ACC',
 'bladder_BLCA',
 'brain_GBM',
 'brain_MISC',
 'breast_BRCA',
 'colon_COLO',
 'esophagus_ESCA',
 'esophagus_HNSC',
 'gall_bladder_CHOL',
 'kidney_KDNY',
 'liver_HCC',
 'lung_LUNG',
 'oral_HNSC',
 'oral_SECR',
 'oral_SKCM',
 'other_MISC',
 'other_SECR',
 'ovary_OV',
 'pancreas_PAAD',
 'parotid_SECR',
 'prostate_PRAD',
 'sarcomatoid_SARC',
 'skin_HNSC',
 'skin_MISC',
 'skin_SECR',
 'skin_SKCM',
 'stomach_STAD',
 'testis_MISC',
 'testis_TGCT',
 'thymus_THYM',
 'thyroid_THCA'}

In [176]:
# met500 
met500_meta = tc.get(name='met500-fc3c', file='met500_meta')
met500_TPM = tc.get(name='met500-fc3c', file='met500_TPM') #20,979x868 matrix

#Novartis_PDX
Novartis_PDX_ann = tc.get(name='pdx-data-3d29', file='Novartis_PDX_ann')
Novartis_PDX_TPM = tc.get(name='pdx-data-3d29', file='Novartis_PDX_TPM').T # 38,087x445

#pediatric_PDX
pediatric_PDX_ann = tc.get(name='pdx-data-3d29', file='pediatric_PDX_ann')
pediatric_PDX_TPM = tc.get(name='pdx-data-3d29', file='pediatric_PDX_TPM') #80,000x250

No dataset version provided. Using version 1.
No dataset version provided. Using version 1.
No dataset version provided. Using version 2.
No dataset version provided. Using version 2.
No dataset version provided. Using version 2.
No dataset version provided. Using version 2.


In [18]:
set(Novartis_PDX_ann.lineage+"_"+Novartis_PDX_ann.subtype)

{'NS_adenocarcinoma',
 'NS_neuroendocrine',
 'autonomic_ganglia_NS',
 'bile_duct_adenocarcinoma',
 'bile_duct_adenosquamous_carcinoma',
 'bone_NS',
 'bone_chondroblastic',
 'bone_dedifferentiated',
 'bone_myxoid',
 'breast_NS',
 'breast_ductal_carcinoma',
 'breast_ductal_carcinoma_in_situ',
 'breast_lobular_carcinoma',
 'breast_metaplastic_carcinoma',
 'central_nervous_system_NS',
 'central_nervous_system_astrocytoma_Grade_IV',
 'central_nervous_system_ependymoma_Grade_III-IV',
 'colorectal_NS',
 'colorectal_adenocarcinoma',
 'colorectal_squamous_cell_carcinoma',
 'esophagus_adenocarcinoma',
 'esophagus_squamous_cell_carcinoma',
 'gastric_adenocarcinoma',
 'gastric_diffuse_adenocarcinoma',
 'kidney_NS',
 'kidney_clear_cell_renal_cell_carcinoma',
 'kidney_transitional_cell_carcinoma',
 'liver_hepatocellular_carcinoma',
 'lung_NS',
 'lung_adenocarcinoma',
 'lung_basaloid_carcinoma',
 'lung_large_cell_carcinoma',
 'lung_mixed_adenosquamous_carcinoma',
 'lung_non_small_cell_carcinoma',
 'l

In [177]:
met500_meta.iloc[0]

Sample_id        ES_5001-capt-SI_5013-C0LAMACXX
sample_type                               tumor
sample_source                           ES_5001
dataset                                    mctp
tissue                                   breast
cohort                                     BRCA
run.id           ES_5001-capt-SI_5013-C0LAMACXX
idx                                         461
test                                      False
tc                                         0.93
biopsy_tissue                             brain
Name: 0, dtype: object

In [178]:
tcga_dict = {
"LAML": "Acute Myeloid Leukemia",
"ACC": "Adrenocortical carcinoma",
"BLCA": "Bladder Urothelial Carcinoma",
"BOCA": "Bone Cancer",
"LGG": "Brain Lower Grade Glioma",
"BRCA": "Breast invasive carcinoma",
"CESC": "Cervical squamous cell carcinoma and endocervical adenocarcinoma",
"CHOL": "Cholangiocarcinoma",
"CLLE": "Chronic Lymphocytic Leukemia",
"CMDI": "Chronic Myeloid Disorders",
"COAD": "Colon adenocarcinoma",
"COLO": "Colorectal Cancer",
"COADREAD": "Colorectal cancer",
"EOPC": "Early Onset Prostate Cancer",
"ESAD": "Esophageal Adenocarcinoma",
"ESCA": "Esophageal carcinoma",
"CHOL": "Gallbladder cancer",
"GBM": "Glioblastoma multiforme",
"HNSC": "Head and Neck squamous cell carcinoma",
"KDNY": "Kidney Cancer",
"KICH": "Kidney Chromophobe",
"KIRC": "Kidney renal clear cell carcinoma",
"KIRP": "Kidney renal papillary cell carcinoma",
"LIRI": "Liver Cancer",
"LICA": "Liver Cancer",
"LINC": "Liver Cancer",
"HCC": "Liver hepatocellular carcinoma",
"LIHC": "Liver hepatocellular carcinoma",
"LGG": "Lower Grade GLioma",
"LUNG": "Lung Cancer",
"LUAD": "Lung adenocarcinoma",
"LUSC": "Lung squamous cell carcinoma",
"DLBC": "Lymphoid Neoplasm Diffuse Large B-cell Lymphoma",
"MCTP": "MCTP",
"MALY": "Malignant Lymphoma",
"MESO": "Mesothelioma",
"NBL": "Neuroblastoma",
"ORCA": "Oral Cancer",
"MISC": "Other Cancer",
"OV": "Ovarian serous cystadenocarcinoma",
"PACA": "Pancreatic Cancer",
"PAEN": "Pancreatic Cancer Endocrine neoplasms",
"PAAD": "Pancreatic adenocarcinoma",
"PBCA": "Pediatric Brain Cancer",
"PCPG": "Pheochromocytoma and Paraganglioma",
"PRAD": "Prostate adenocarcinoma",
"READ": "Rectum adenocarcinoma",
"RECA": "Renal Cancer",
"SARC": "Sarcoma",
"SECR": "Secretory Cancer",
"SKCM": "Skin Cutaneous Melanoma",
"STAD": "Stomach adenocarcinoma",
"TGCT": "Testicular Germ Cell Tumor",
"TGCT": "Testicular Germ Cell Tumors",
"THYM": "Thymoma",
"THYM": "Thymoma",
"THCA": "Thyroid carcinoma",
"UCS":  "Uterine Carcinosarcoma",
"UCEC": "Uterine Corpus Endometrial Carcinoma",
"UVM": "Uveal Melanoma",
"ACC": "adrenocortical carcinoma",
}

In [179]:
met500_meta = met500_meta.replace({"cohort": tcga_dict})

In [180]:
set(met500_meta.columns) - set(rename.values())

{'Sample_id',
 'biopsy_tissue',
 'cohort',
 'idx',
 'run.id',
 'sample_source',
 'tc',
 'test',
 'tissue'}

In [181]:
met500_meta['sequencer'] = "Illumina HiSeq 2000"
met500_meta['method'] = "bulk"
met500_meta['cell_type'] = "tumor"
met500_meta['reference']= "hg38"
met500_meta['ends']="paired end"
met500_meta['metastatic/primary'] = "metastatic"
met500_meta['dataset'] = "met500"

In [182]:
met500_meta

,Sample_id,sample_type,sample_source,dataset,tissue,cohort,run.id,idx,test,tc,biopsy_tissue,sequencer,method,cell_type,reference,ends,metastatic/primary
0,ES_5001-capt-SI_5013-C0LAMACXX,tumor,ES_5001,met500,breast,Breast invasive carcinoma,ES_5001-capt-SI_5013-C0LAMACXX,461,False,0.93,brain,Illumina HiSeq 2000,bulk,tumor,hg38,paired end,metastatic
1,ES_5004-capt-SI_5834-C19KEACXX,tumor,ES_5004,met500,colon,Colorectal Cancer,ES_5004-capt-SI_5834-C19KEACXX,468,False,0.47,soft_tissue,Illumina HiSeq 2000,bulk,tumor,hg38,paired end,metastatic
2,ES_5004-poly-SI_5767-C19KEACXX,tumor,ES_5004,met500,colon,Colorectal Cancer,ES_5004-poly-SI_5767-C19KEACXX,469,False,0.47,soft_tissue,Illumina HiSeq 2000,bulk,tumor,hg38,paired end,metastatic
3,ES_5005-capt-SI_5505-D130HACXX,tumor,ES_5005,met500,other,Other Cancer,ES_5005-capt-SI_5505-D130HACXX,470,False,0.41,cervix,Illumina HiSeq 2000,bulk,tumor,hg38,paired end,metastatic
4,ES_5005-poly-SI_5486-D12YGACXX,tumor,ES_5005,met500,other,Other Cancer,ES_5005-poly-SI_5486-D12YGACXX,471,False,0.41,cervix,Illumina HiSeq 2000,bulk,tumor,hg38,paired end,metastatic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,TP_2123-poly-SI_11689-C7G60ANXX,tumor,TP_2123,met500,brain,Other Cancer,TP_2123-poly-SI_11689-C7G60ANXX,4154,False,0.52,brain,Illumina HiSeq 2000,bulk,tumor,hg38,paired end,metastatic
864,TP_2130-capt-SI_11905-C7FMDANXX,tumor,TP_2130,met500,None,Head and Neck squamous cell carcinoma,TP_2130-capt-SI_11905-C7FMDANXX,4160,True,0.46,oral,Illumina HiSeq 2000,bulk,tumor,hg38,paired end,metastatic
865,TP_2131-capt-SI_11906-C7F4VANXX,tumor,TP_2131,met500,bladder,Bladder Urothelial Carcinoma,TP_2131-capt-SI_11906-C7F4VANXX,4161,False,0.43,liver,Illumina HiSeq 2000,bulk,tumor,hg38,paired end,metastatic
866,TP_2141-capt-SI_12056-H53C5ADXX,tumor,TP_2141,met500,breast,Breast invasive carcinoma,TP_2141-capt-SI_12056-H53C5ADXX,4169,False,0.67,liver,Illumina HiSeq 2000,bulk,tumor,hg38,paired end,metastatic


In [183]:
met500_meta = met500_meta.rename(columns={**rename, **{'subtype': "disease_type"}})
met500_meta = met500_meta[set(rename.values()) & set(met500_meta.columns)].set_index('sample_id', drop=True)

In [165]:
ann = pd.concat([met500_meta, CCLE_annotation])
ann

,participant_id,tissue_type,disease_type,cell_type,tumor_purity,dataset,collection_site,method,sample_type,ends,...,reference,media_type,age,sequencing_type,disease_subtype,sex,sample_id,hasebv,tissue_id,doublingt
ES_5001-capt-SI_5013-C0LAMACXX,ES_5001,breast,Breast invasive carcinoma,tumor,0.93,met500,brain,bulk,tumor,paired end,...,hg38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ES_5004-capt-SI_5834-C19KEACXX,ES_5004,colon,Colorectal Cancer,tumor,0.47,met500,soft_tissue,bulk,tumor,paired end,...,hg38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ES_5004-poly-SI_5767-C19KEACXX,ES_5004,colon,Colorectal Cancer,tumor,0.47,met500,soft_tissue,bulk,tumor,paired end,...,hg38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ES_5005-capt-SI_5505-D130HACXX,ES_5005,other,Other Cancer,tumor,0.41,met500,cervix,bulk,tumor,paired end,...,hg38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ES_5005-poly-SI_5486-D12YGACXX,ES_5005,other,Other Cancer,tumor,0.41,met500,cervix,bulk,tumor,paired end,...,hg38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CDS-qbHB8J,PT-kiG6X0x0,lymphocyte,Lymphoma,historical_CL; 2D,NaN,ccle,pleural_effusion,bulk,NaN,paired end,...,hg38,"RPMI 1640 + 10% FBS\n, Suspension with aggregates",73,NaN,non_hodgkin_lymphoma,Female,SUDHL2,False,ACH-002498,NaN
CDS-y7yI24,PT-9Sjq1s2c,skin,Skin Cancer,historical_CL; 2D,NaN,ccle,colon,bulk,NaN,paired end,...,hg38,NaN,Adult,NaN,melanoma,Male,VMRCMELG,False,ACH-001704,60.0
CDS-cteUkO,PT-55Pt5qaS,lymphocyte,Lymphoma,historical_CL; 2D,NaN,ccle,pleural_effusion,bulk,NaN,paired end,...,hg38,NaN,46,NaN,lymphoma_unspecified,Female,WSUNHL,False,ACH-001709,57.0
CDS-dQKiht,PT-TRbh2Ida,breast,normal,historical_CL; 2D,NaN,ccle,breast,bulk,NaN,paired end,...,hg38,NaN,52,NaN,normal,Female,HCC1143_NORMAL,NaN,ACH-003020,NaN


In [166]:
set(CCLE_expression.index) & set(met500_meta.index)

set()

In [186]:
len(set(CCLE_expression.columns) - set(met500_TPM.columns)), len(set(CCLE_expression.columns) & set(met500_TPM.columns)), len(set(met500_TPM.columns) - set(CCLE_expression.columns)),

(33199, 18855, 2124)

In [187]:
pediatric_PDX_ann.iloc[0]

sampleID                                                                                  ALL-102
lineage                                                                                     blood
subtype                                                                                       ALL
Histology                                                                                     ALL
Histology.Detailed                                                                     Ph-likeALL
Histology-Detailed2                                                                    Ph-likeALL
Molecular-Subtype-Brain                                                                      None
PI                                                                                           Lock
Sex                                                                                          Male
Phase                                                                                   Diagnosis
Age                 

In [188]:
[(i.split('me patient as ')[-1].split(' (')[0],v) if type(i) is str and 'ame patient' in i else '' for v, i in pediatric_PDX_ann[["sampleID","Other_info1"]].values]

[('ALL-105', 'ALL-102'),
 ('ALL-102', 'ALL-105'),
 '',
 '',
 '',
 '',
 ('ALL-102', 'ALL-115'),
 '',
 '',
 '',
 ('ALL-46', 'ALL-121'),
 '',
 ('ALL-58', 'ALL-123'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('ALL-121', 'ALL-46'),
 '',
 '',
 '',
 '',
 '',
 ('ALL-123', 'ALL-58'),
 '',
 ('ALL-25', 'ALL-61'),
 '',
 '',
 ('ALL-81', 'ALL-80'),
 ('ALL-80', 'ALL-81'),
 '',
 ('ALL-32', 'ALL-90'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('ALL-61', 'ALL-25'),
 '',
 '',
 '',
 ('ALL-90', 'ALL-32'),
 '',
 '',
 '',
 '',
 '',
 '',
 ('ALL-83', 'ALL-82'),
 ('ALL-82', 'ALL-83'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('COG-N-453x', 'COG-N-452x'),
 ('COG-N-452x', 'COG-N-453x'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('COG-N-623x', 'COG-N-603x'),
 ('COG-N-618x', 'COG-N-619x'),
 '',
 ('COG-N-603x', 'COG-N-623x'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('98

In [189]:
pediatric_PDX_ann['participant_id'] = pediatric_PDX_ann.index

#created frrom manual inspection
samepatient = [('NCH-CA-2', 'NCH-CA-1'), ('ALL-105', 'ALL-102', "ALL-115"), ('ALL-46', 'ALL-121'), ('ALL-25', 'ALL-61'), ('ALL-81', 'ALL-80'), ('ALL-32', 'ALL-90'), ('ALL-58', 'ALL-123'), ('ALL-82', 'ALL-83'), ("COG-N-623x", "COG-N-603x"), ("COG-N-453x","COG-N-452x"), ("COG-N-618x", "COG-N-619x"), ('22909PNET', '9850PNET'), ('OS-34', 'OS-34-SJ'), ('OS-36', 'OS-36-SJ', 'OS-32'),  ('Rh-30R', 'Rh-30')]

for val in samepatient:
    for i in val[1:]:
        pediatric_PDX_ann.loc[i, 'participant_id']=val[0]
        
pediatric_PDX_ann['age'] = ['adult' if i =='Adult' else 'child' for i in pediatric_PDX_ann['Other_info1']]

In [190]:
pediatric_PDX_ann = pediatric_PDX_ann.rename(columns={**rename, **{'subtype': "disease_type"}})
print(set(pediatric_PDX_ann.columns)-set(rename.values()))
#pediatric_PDX_ann = pediatric_PDX_ann[set(rename.values()) & set(pediatric_PDX_ann.columns)].set_index('sample_id', drop=True)

{'Site.of.Initial.Tumor', 'Risk.Group', 'PC1', 'Patient.last.alive.year', 'Prior Therapy', 'Histology.Detailed', 'Time.from.Dx.to.sample.for.model.days.', 'PC3', 'New Histopathology', 'Histology-Detailed2', 'PC2', 'Other_info1', 'Histology', 'Reported_Ethnicity', 'Patient.EFS.from.Dx.to.1st.Progression.days.', 'Patient.OS.to.last.alive.date.days.', 'Stage.of.Disease', 'PI', 'Molecular-Subtype-Brain', 'COG studies', 'Site.of.Specimen'}


In [191]:
pediatric_PDX_ann['sequencer'] = "Illumina HiSeq 2000"
pediatric_PDX_ann['align'] = "BWA"
pediatric_PDX_ann['method'] = "bulk"
pediatric_PDX_ann['reference']= "hg19"
pediatric_PDX_ann['ends']="paired end"
pediatric_PDX_ann['dataset'] = "pediatric_PDX"

In [164]:
met500_meta = met500_meta.rename(columns={**rename, **{'subtype': "disease_type"}})[set(rename.values()) & set(met500_meta.columns)].set_index('sample_id', drop=True)

In [165]:
ann = pd.concat([met500_meta, CCLE_annotation])
ann

,participant_id,tissue_type,disease_type,cell_type,tumor_purity,dataset,collection_site,method,sample_type,ends,...,reference,media_type,age,sequencing_type,disease_subtype,sex,sample_id,hasebv,tissue_id,doublingt
ES_5001-capt-SI_5013-C0LAMACXX,ES_5001,breast,Breast invasive carcinoma,tumor,0.93,met500,brain,bulk,tumor,paired end,...,hg38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ES_5004-capt-SI_5834-C19KEACXX,ES_5004,colon,Colorectal Cancer,tumor,0.47,met500,soft_tissue,bulk,tumor,paired end,...,hg38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ES_5004-poly-SI_5767-C19KEACXX,ES_5004,colon,Colorectal Cancer,tumor,0.47,met500,soft_tissue,bulk,tumor,paired end,...,hg38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ES_5005-capt-SI_5505-D130HACXX,ES_5005,other,Other Cancer,tumor,0.41,met500,cervix,bulk,tumor,paired end,...,hg38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ES_5005-poly-SI_5486-D12YGACXX,ES_5005,other,Other Cancer,tumor,0.41,met500,cervix,bulk,tumor,paired end,...,hg38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CDS-qbHB8J,PT-kiG6X0x0,lymphocyte,Lymphoma,historical_CL; 2D,NaN,ccle,pleural_effusion,bulk,NaN,paired end,...,hg38,"RPMI 1640 + 10% FBS\n, Suspension with aggregates",73,NaN,non_hodgkin_lymphoma,Female,SUDHL2,False,ACH-002498,NaN
CDS-y7yI24,PT-9Sjq1s2c,skin,Skin Cancer,historical_CL; 2D,NaN,ccle,colon,bulk,NaN,paired end,...,hg38,NaN,Adult,NaN,melanoma,Male,VMRCMELG,False,ACH-001704,60.0
CDS-cteUkO,PT-55Pt5qaS,lymphocyte,Lymphoma,historical_CL; 2D,NaN,ccle,pleural_effusion,bulk,NaN,paired end,...,hg38,NaN,46,NaN,lymphoma_unspecified,Female,WSUNHL,False,ACH-001709,57.0
CDS-dQKiht,PT-TRbh2Ida,breast,normal,historical_CL; 2D,NaN,ccle,breast,bulk,NaN,paired end,...,hg38,NaN,52,NaN,normal,Female,HCC1143_NORMAL,NaN,ACH-003020,NaN


In [166]:
set(CCLE_expression.index) & set(met500_meta.index)

set()

In [137]:
cclfrna.columns = [i.split('(')[-1][:-1] if '(' in i else i for i in cclfrna.columns]

In [139]:
len(set(CCLE_expression.columns) - set(cclfrna.columns)), len(set(CCLE_expression.columns) & set(cclfrna.columns)), len(set(cclfrna.columns) - set(CCLE_expression.columns)),

(6843, 45211, 10)

In [350]:
Novartis_PDX_ann.iloc[0]

sampleID           0931HXXTM
lineage             pancreas
subtype     ductal_carcinoma
type                     PDX
Name: 0, dtype: object

In [ ]:
Novartis_PDX_ann = Novartis_PDX_ann.rename(columns=rename).set_index('sample_id', drop=True)

In [ ]:
Novartis_PDX_ann['sequencer'] = "Illumina HiSeq 2000"
Novartis_PDX_ann['method'] = "bulk"
Novartis_PDX_ann['align'] = "STAR"
Novartis_PDX_ann['reference']= "hg19"
Novartis_PDX_ann['ends']="paired end"
Novartis_PDX_ann['dataset'] = "Novartis_PDX"

## tumor inf elife

In [13]:
elife_tumorinf = tc.get(name='tumor-infiltration-3307', version=1, file='elife_tumorinf')

In [14]:
elife_tumorinf

,Bcells,CAFs,CD4_Tcells,CD8_Tcells,Endothelial,Macrophages,NKcells
A1BG,7.75210,4.47890,4.6702,5.02000,0.0000,3.46370,1.74260
A1BG-AS1,0.27272,28.98800,6.7471,11.76400,0.0000,2.36580,8.92140
A1CF,0.94832,0.03625,0.4978,0.56538,0.2582,0.13695,0.25936
A2M,0.14435,184.45000,2.9531,15.38600,1575.2000,418.67000,8.94110
A2M-AS1,0.00000,0.34500,7.3420,11.59100,8.6042,0.35669,0.21549
...,...,...,...,...,...,...,...
ZYG11A,13.37900,3.64120,18.2580,18.17900,9.0574,7.47320,16.04700
ZYG11B,18.64800,10.59500,15.2930,13.80700,5.1119,5.00460,8.10280
ZYX,31.07300,205.32000,79.5490,64.75700,70.3870,206.18000,102.91000
ZZEF1,5.69700,2.98250,9.9508,14.66500,7.2201,5.78570,16.66700


In [ ]:
elife_tumorinf = elife_tumorinf.rename(columns={"Bcells": "B-cell", "CAFs": "CAF", "CD4_Tcells": "CD4_T-cells", "CD8_Tcells": "CD8_T-cells","macrophage": "macrophage", "Endothelial": "endothelial", "NKcells": "NK-cell"})

In [13]:
elife_tumorinf_ann = pd.DataFrame()
elife_tumorinf_ann["cell_type"] = "normal"
elife_tumorinf_ann["tissue_type"] = elife_tumorinf.columns
elife_tumorinf_ann["sample_ID"] = elife_tumorinf.columns
elife_tumorinf_ann['sequencer'] = ""
elife_tumorinf_ann['align'] = "bowtie"
elife_tumorinf_ann['method'] = "singlecell"
elife_tumorinf_ann['reference']= "hg19"
LAU125	59	male	iliac lymph node
LAU355	70	female	iliac-obturator lymph node
LAU1255	87	male	axillary lymph node
LAU1314	81	male	iliac-obturator lymph node
elife_tumorinf_ann['age'] = ""
elife_tumorinf_ann['sex'] = ""
elife_tumorinf_ann['dataset'] = "elife_tumorinf"

## tirosh's melanoma

In [4]:
melanoma = tc.get(name='tirosh-melanoma-scrnaseq-60f0', file='melanoma')

No dataset version provided. Using version 1.


In [19]:
melanoma.columns = [i.replace('-', '_').replace('Cy', "CY").replace('cy', "CY").replace('CY88C', 'CY88_C').replace('CY89A', "CY89_A").replace('CY89C', 'CY89_C').replace('CY89F', 'CY89_F').replace('CY89N', 'CY89_N').replace('CY94C', 'CY94_C') for i in melanoma.columns]

In [20]:
melanoma_ann = pd.DataFrame()

typ={1:"normal", 2:"tumor",0: np.nan}
orig={1:"melanoma", 2:"B-cell", 3: "macrophage", 4: "endothelial", 5: "CAF", 6:"NK-cell", 0: np.nan}

melanoma_ann['age'] = [int(i) for i in melanoma.loc['tumor']]
melanoma_ann["cell_type"] = [typ[int(i)] for i in melanoma.loc['malignant(1=no,2=yes,0=unresolved)']]
melanoma_ann['tissue_type'] = [orig[int(i)] for i in melanoma.loc['non-malignant cell type (1=T,2=B,3=Macro.4=Endo.,5=CAF;6=NK)']]
melanoma_ann['name'] = [i.split('_')[0] for i in melanoma.columns]
melanoma_ann['sample_id'] = melanoma.columns
melanoma_ann['other'] = [i.split('_')[-2] for i in melanoma.columns]
melanoma_ann['sequencer'] = ""
melanoma_ann['method'] = "singlecell"

melanoma_ann['reference']= "hg38"
melanoma_ann['ends']="paired end"
melanoma_ann['dataset'] = "melanoma"

## GTEX

In [9]:
#! curl https://storage.googleapis.com/gtex_analysis_v9/snrna_seq_data/GTEx_8_tissues_snRNAseq_atlas_071421.public_obs.h5ad --output temp/gtex_8_atlas_public.h5ad

## GTEX additional
https://storage.googleapis.com/gtex_external_datasets/eyegex_data/rna_seq_data/EyeGEx_retina_combined_genelevel_expectedcounts_byrid_nooutlier.tpm.matrix.gct
    
https://storage.googleapis.com/gtex_external_datasets/eyegex_data/annotations/EyeGEx_meta_combined_inferior_retina_summary_deidentified_geo_ids.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1810M  100 1810M    0     0   126M      0  0:00:14  0:00:14 --:--:--  126M


In [614]:
gtex_v9 = read_h5ad("temp/gtex_8_atlas_public.h5ad") #209,126 × 17,695

In [623]:
gtex_v9.obs = gtex_v9.obs[["Age_bin","Sex","Sample ID", "Participant ID", "RIN score from PAXgene tissue Aliquot", "Tissue", "tissue"]].rename(columns={**rename, **{"tissue": "tissue_type", "Tissue": "collection_site"}})

n_genes                                                                    2658
fpr                                                                         0.1
tissue                                                           skeletalmuscle
prep                                                                        CST
individual                                                                   01
nGenes                                                                     2902
nUMIs                                                                   11544.0
PercentMito                                                             0.07623
PercentRibo                                                            0.051195
Age_bin                                                                   51-60
Sex                                                                        Male
Sample ID                                              GTEX-1HSMQ-5011-SM-GKSJH
Participant ID                          

In [ ]:
gtex_v9.obs['sequencer']="Illumina HiSeq 2000"
gtex_v9.obs['method']="bulk"
gtex_v9.obs['reference']= "hg38"
gtex_v9.obs['ends']="paired end"
gtex_v9.obs['dataset']="gtex"

In [ ]:
gtex_add = # 80,000 x 500

## THEIS LAB scRNAseq datasets

In [ ]:
https://theislab.github.io/sfaira-portal/Datasets #50,000x13,000


## HCMI

In [9]:
# HCMI dataset
# Code to generate this dataset can be found here:
# https://github.com/broadinstitute/hcmi-processing/blob/main/hcmi-rna-analysis-210226.ipynb
hcmi_ltpm = tc.get(name='hcmi-data-ac4b', file='hcmi_ltpm').T # 60486 x 157
hcmi_sample_info = tc.get(name='hcmi-data-ac4b', file='hcmi_sample_info')
#sample_info = tc.get(name='hcmi-data-ac4b', file='sample-info')

No dataset version provided. Using version 8.
Received a not found error. Are you sure about your credentials and/or the data parameters? params: {'dataset_permaname': 'hcmi-data-ac4b', 'version': '8', 'datafile_name': 'hcmi_ltpm', 'format': 'metadata', 'taigapy_version': '3.3.2'}


AttributeError: 'NoneType' object has no attribute 'T'

In [7]:
hcmi_sample_info['type'] = [i.split('_')[1] +"; "+str(j) for i,j in hcmi_sample_info[['type', 'Type']].values]

In [8]:
hcmi_sample_info[['Case ID', 'Clinical Tumor Diagnosis', 'subtype', 'Tissue Status', "Acquisition Site", 'Gender', 'Race', 'Age At Acquisition (Years)', 'Expansion Status', 'sampleID', 'type', 'lineage']].rename(columns=rename)

Case ID                                                                  HCM-CSHL-0092-C25
Primary Site                                                                      Pancreas
Clinical Tumor Diagnosis                                                 Pancreatic cancer
subtype                                                         Adenocarcinoma ductal type
Tissue Status                                                                      Primary
Acquisition Site                                                           Pancreatic head
Gender                                                                                Male
Race                                                                               Unknown
Age At Acquisition (Years)                                                            69.0
Age At Diagnosis (Years)                                                              69.0
Disease Status                                                         Progressive disease

In [ ]:
hcmi_sample_info['sequencer'] =""
hcmi_sample_info['method'] =""
hcmi_sample_info['reference']= ""
hcmi_sample_info['ends']=""
hcmi_sample_info['dataset']="hcmi"

## L1000 dataset

In [4]:
import subprocess
from anndata import AnnData 

In [25]:
# you will need R > 4.0 
# https://www.charlesbordet.com/en/how-to-upgrade-to-R-4-0-0-on-debian/#the-naive-solution
! R -e "if(!requireNamespace('BiocManager', quietly = TRUE)){install.packages('BiocManager', repos='http://cran.us.r-project.org')};BiocManager::install('cmapR');"


R version 4.1.2 (2021-11-01) -- "Bird Hippie"
Copyright (C) 2021 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> if(!requireNamespace("BiocManager", quietly = TRUE)){install.packages("BiocManager", repos="http://cran.us.r-project.org")};BiocManager::install(c("limma"));
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Warning in install.packages("BiocManager", repos = "http://cran.us.r-pr

In [31]:
folder = "gs://ccle_default_params/celligner_ex/"
res = []
for val in ["level5_beta_ctl_n58022x12328.gctx",
            "level5_beta_trt_cp_n720216x12328.gctx",
            "level5_beta_trt_misc_n8283x12328.gctx"
            "level5_beta_trt_oe_n34171x12328.gctx",
            "level5_beta_trt_sh_n238351x12328.gctx",
            "level5_beta_trt_xpr_n142901x12328.gctx",]:
    cmd = "gsutil cp " + folder + val + " temp/"
    ! $cmd
    res.append(h.loadGCTXasAnnData('temp/'+val))

Copying gs://ccle_default_params/celligner_ex/level5_beta_trt_misc_n8283x12328.gctx...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").


Operation completed over 1 objects/389.6 MiB.                                    


NameError: name 'pg' is not defined

In [ ]:
ann['sequencer'] = "L1000"
ann['method'] = "L1000"
ann['reference']= "L1000"
ann['ends']="L1000"
ann['dataset']="L1000"

## encode

In [136]:
todl = h.fileToList('data/encode_rna.txt')

In [134]:
# 40,000 x 1100
report = pd.read_csv('data/encode_report.tsv', sep="\t", skiprows=1)
report = report[report.columns[report.isna().sum()!=len(report)]]
report

,ID,Accession,Assay name,Assay title,Biosample summary,Biosample term name,Dbxrefs,Description,Lab,Project,...,Biosample treatment,Biosample treatment ontology ID,Biosample treatment amount,Biosample treatment amount units,Biosample treatment duration,Biosample treatment duration units,Biosample modification site target organism,Replicates,Cellular component,Library construction method
0,/experiments/ENCSR620LQN/,ENCSR620LQN,RNA-seq,total RNA-seq,Homo sapiens esophagus muscularis mucosa tissu...,esophagus muscularis mucosa,GEO:GSE88409,The libraries contained in this Experiment com...,"Thomas Gingeras, CSHL",ENCODE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/replicates/499e1412-5ef3-4ac2-98be-fd55bb01fad5/,NaN,NaN
1,/experiments/ENCSR406SAW/,ENCSR406SAW,RNA-seq,total RNA-seq,Homo sapiens upper lobe of left lung tissue fe...,upper lobe of left lung,GEO:GSE88254,The libraries contained in this Experiment com...,"Thomas Gingeras, CSHL",ENCODE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/replicates/782b72f8-3197-4a66-951c-e2de88158b6a/,NaN,NaN
2,/experiments/ENCSR019MXZ/,ENCSR019MXZ,polyA plus RNA-seq,polyA plus RNA-seq,Homo sapiens HepG2 insoluble cytoplasmic fraction,HepG2,GEO:GSE87958,Initial insoluble fractions on HepG2 Long Poly...,"Eric Lécuyer, IRCM",ENCODE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/replicates/443af8c7-5ed1-4930-b4f7-e1ca62c553...,insoluble cytoplasmic fraction,NaN
3,/experiments/ENCSR630VJN/,ENCSR630VJN,RNA-seq,total RNA-seq,Homo sapiens transverse colon tissue male adul...,transverse colon,GEO:GSE88418,The libraries contained in this Experiment com...,"Thomas Gingeras, CSHL",ENCODE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/replicates/aa7ff7b6-5bf5-4f0d-a09c-8612c945df37/,NaN,NaN
4,/experiments/ENCSR035SKV/,ENCSR035SKV,RNA-seq,total RNA-seq,Homo sapiens gastroesophageal sphincter tissue...,gastroesophageal sphincter,GEO:GSE87978,The libraries contained in this Experiment com...,"Thomas Gingeras, CSHL",ENCODE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/replicates/8c825033-47e6-4659-8b02-0b399fab0435/,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006,/experiments/ENCSR672JUF/,ENCSR672JUF,polyA plus RNA-seq,polyA plus RNA-seq,Homo sapiens foreskin keratinocyte male newborn,foreskin keratinocyte,GEO:GSM958177,NaN,"Joseph Costello, UCSF",Roadmap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/replicates/dd125c50-da07-473b-90cf-6be56b7fe968/,NaN,NaN
1007,/experiments/ENCSR999CPT/,ENCSR999CPT,polyA plus RNA-seq,polyA plus RNA-seq,Homo sapiens placental basal plate tissue fema...,placental basal plate,GEO:GSM1127098,NaN,"Joseph Costello, UCSF",Roadmap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/replicates/d92f45a2-c13f-42d4-a52d-2f974f6e7f41/,NaN,NaN
1008,/experiments/ENCSR634LOX/,ENCSR634LOX,polyA plus RNA-seq,polyA plus RNA-seq,Homo sapiens foreskin melanocyte male newborn,foreskin melanocyte,GEO:GSM958174,NaN,"Joseph Costello, UCSF",Roadmap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/replicates/f0421981-9298-4ade-a256-59aa55666f27/,NaN,NaN
1009,/experiments/ENCSR714QAF/,ENCSR714QAF,polyA plus RNA-seq,polyA plus RNA-seq,Homo sapiens mole tissue female,mole,GEO:GSM1582478,The RNA-Seq libraries contained in this experi...,"Joseph Costello, UCSF",Roadmap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/replicates/919a47f8-b23e-431c-9ea0-9dcb6990df63/,NaN,NaN


In [137]:
region = []
platform = []

for val in list(report['Description']):
    if val is np.nan:
        region.append('')
        platform.append('')
        continue
    if "cytosol" in val or "cytosolic" in val:
        region.append('cytosol')
    elif "nucleolus" in val:
        region.append('nucleolus')
    elif "nucleus" in val:
        region.append('nucleus')
    elif "whole cell" in val:
        region.append('whole')
    elif "chromatin" in val:
        region.append('chromatin')
    elif "cytoplasmic" in val:
        region.append('cytoplasm')
    else:
        region.append(None)
    if "Hi-Seq" in val:
        platform.append('illumina Hi-Seq 2000')
    elif "Illumina GAIIx" in val:
        platform.append('illumina gaiix')
    else:
        platform.append('')

In [138]:
report.loc[report['Cellular component'].isna(), 'Cellular component'] = np.array(region)[report[report['Cellular component'].isna()].index]

In [139]:
report['sequencer'] = platform

In [140]:
report['Files'] = report['Files'].str.split(',').apply(lambda x: [i.split('/')[-2] for i in x])

In [141]:
report = report[report['Cellular component'].isna() | (report['Cellular component']=="nucleus")]
report = report[['Assay title', 'Biosample summary', 'Biosample term name', 'Description', 'Lab', 'Project', 'Files', 'Biosample accession', 'Organism', 'Life stage', 'Biosample age', 'sequencer', 'Biosample treatment ontology ID', 'Biosample treatment amount',
'Biosample treatment amount units', 'Biosample treatment duration',
'Biosample treatment duration units']].rename(columns=rename)

NameError: name 'rename' is not defined

In [145]:
set(report.platform)

{'', 'illumina Hi-Seq 2000', 'illumina gaiix'}

In [ ]:
match = {}
for i, val in report['Files'].iteritems():
    for j in val:
        match[j] = i

In [ ]:
encode_ann = pd.DataFrame(columns=report.columns)
for val in todl:
    val = val.split('/')[-1].split('.')[0]
    if val not in match:
        print(val)
    else:
        encode_ann.loc[val] = report.loc[match[val]].values

In [ ]:
encode_ann['sequencer'] = ""# MISSING
encode_ann['reference']= "hg38"
encode_ann['ends']="paired-end"
encode_ann['align'] = "STAR"
encode_ann['dataset']="encode"

In [17]:
res = []
for val in encode_ann.index:
    vl = pd.read_csv('https://www.encodeproject.org/files/'+val+'/@@download/'+val+'.tsv', sep='\t')
    try:
        vl = vl.set_index('gene_id', drop=True)['TPM'].rename(val)
    except KeyError:
        vl = vl.set_index('gene_ID', drop=True)[vl.columns[-1]].rename(val)
    res.append(vl)
encode = pd.concat(res, axis=1)

ValueError: cannot reindex from a duplicate axis

## ICGC

In [70]:
icgclist = ["https://dcc.icgc.org/api/v1/download?fn=/current/Projects/BOCA-FR/exp_seq.BOCA-FR.tsv.gz",
"https://dcc.icgc.org/api/v1/download?fn=/current/Projects/BPLL-FR/exp_seq.BPLL-FR.tsv.gz",
"https://dcc.icgc.org/api/v1/download?fn=/current/Projects/BRCA-KR/exp_seq.BRCA-KR.tsv.gz",
"https://dcc.icgc.org/api/v1/download?fn=/current/Projects/LICA-FR/exp_seq.LICA-FR.tsv.gz",
"https://dcc.icgc.org/api/v1/download?fn=/current/Projects/LIRI-JP/exp_seq.LIRI-JP.tsv.gz",
"https://dcc.icgc.org/api/v1/download?fn=/current/Projects/ORCA-IN/exp_seq.ORCA-IN.tsv.gz",
"https://dcc.icgc.org/api/v1/download?fn=/current/Projects/OV-AU/exp_seq.OV-AU.tsv.gz",
"https://dcc.icgc.org/api/v1/download?fn=/current/Projects/PACA-AU/exp_seq.PACA-AU.tsv.gz",
"https://dcc.icgc.org/api/v1/download?fn=/current/Projects/PACA-CA/exp_seq.PACA-CA.tsv.gz",
"https://dcc.icgc.org/api/v1/download?fn=/current/Projects/PRAD-CA/exp_seq.PRAD-CA.tsv.gz",
"https://dcc.icgc.org/api/v1/download?fn=/current/Projects/PRAD-FR/exp_seq.PRAD-FR.tsv.gz",
"https://dcc.icgc.org/api/v1/download?fn=/current/Projects/CLLE-ES/exp_seq.CLLE-ES.tsv.gz",
"https://dcc.icgc.org/api/v1/download?fn=/current/Projects/MALY-DE/exp_seq.MALY-DE.tsv.gz",
"https://dcc.icgc.org/api/v1/download?fn=/current/Projects/PAEN-AU/exp_seq.PAEN-AU.tsv.gz",
"https://dcc.icgc.org/api/v1/download?fn=/current/Projects/RECA-EU/exp_seq.RECA-EU.tsv.gz"]

In [69]:
res = []
for val in icgc:
    val = pd.read_csv(val, sep='\t')
    print(set(val['platform']), set(val['gene_model']), set(val['experimental_protocol']), set(val['assembly_version']), set(val['alignment_algorithm']), set(val['normalization_algorithm']))
    res.append(pd.concat([val.loc[val['icgc_sample_id']==i, ['gene_id', 'normalized_read_count']].set_index('gene_id').rename(columns={'normalized_read_count':i}) for i in list(set(val['icgc_sample_id']))], axis=1))
icgc = pd.concat(res, axis=1)

'download\\?fn\\=/current/Projects/BOCA-FR/exp_seq.BOCA-FR.tsv.gz'

In [49]:
donor = pd.read_csv("data/donor.tsv", sep="\t", index_col=0)
donor.columns

Index(['project_code', 'study_donor_involved_in', 'submitted_donor_id',
       'donor_sex', 'donor_vital_status', 'disease_status_last_followup',
       'donor_relapse_type', 'donor_age_at_diagnosis',
       'donor_age_at_enrollment', 'donor_age_at_last_followup',
       'donor_relapse_interval', 'donor_diagnosis_icd10',
       'donor_tumour_staging_system_at_diagnosis',
       'donor_tumour_stage_at_diagnosis',
       'donor_tumour_stage_at_diagnosis_supplemental', 'donor_survival_time',
       'donor_interval_of_last_followup', 'prior_malignancy',
       'cancer_type_prior_malignancy', 'cancer_history_first_degree_relative'],
      dtype='object')

In [51]:
more = pd.read_csv('data/donors_more_csv', index_col=0)
donor.loc[more.index, 'Primary Site'] = more['Primary Site']

In [52]:
donor = donor[["donor_sex", "donor_relapse_type", "donor_age_at_enrollment", "donor_diagnosis_icd10", "donor_tumour_stage_at_diagnosis", 'Primary Site']]

In [53]:
specimen = pd.read_csv("data/specimen.tsv", sep="\t", index_col=0)
specimen.iloc[0]

project_code                                                                     CLLE-ES
study_specimen_involved_in                                                           NaN
submitted_specimen_id                                                      060-0123-01TD
icgc_donor_id                                                                    DO51966
submitted_donor_id                                                                    60
specimen_type                          Primary tumour - blood derived (peripheral blood)
specimen_type_other                                                                  NaN
specimen_interval                                                                 3586.0
specimen_donor_treatment_type                                               no treatment
specimen_donor_treatment_type_other                                                  NaN
specimen_processing                                                                fresh
specimen_processing_o

In [54]:
specimen = specimen[["icgc_donor_id", "specimen_type", "specimen_storage", "specimen_processing"]]

In [55]:
for i, val in specimen.iterrows():
    specimen.loc[i, donor.columns] = donor.loc[val.icgc_donor_id].values 

In [59]:
sample = pd.read_csv("data/sample.tsv", sep="\t", index_col=0)
sample

,project_code,submitted_sample_id,icgc_specimen_id,submitted_specimen_id,icgc_donor_id,submitted_donor_id,analyzed_sample_interval,percentage_cellularity,level_of_cellularity,study
icgc_sample_id,,,,,,,,,,
SA564770,CLLE-ES,060-0123-01TD,SP130360,060-0123-01TD,DO51966,60,315.0,97.0,NaN,NaN
SA564151,CLLE-ES,060-01-2ND,SP130410,060-01-2ND,DO51966,60,NaN,98.0,NaN,NaN
SA602505,CLLE-ES,060-0123-02ND,SP199735,060-0123-02ND,DO51966,60,NaN,NaN,NaN,NaN
SA564150,CLLE-ES,060-02-01ND,SP130409,060-02-01ND,DO51966,60,NaN,99.0,NaN,NaN
SA538928,CLLE-ES,060-0123-03TR,SP114999,060-0123-03TR,DO51966,60,315.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
SA607197,BPLL-FR,14_3,SP202799,B-PLL_32_tumor,DO233963,B-PLL_32,NaN,NaN,NaN,NaN
SA607150,BPLL-FR,14_2,SP202986,B-PLL_32_control,DO233963,B-PLL_32,NaN,NaN,NaN,NaN
SA538993,CLLE-ES,356-01-8ND,SP115086,356-01-8ND,DO7084,356,NaN,100.0,NaN,PCAWG


In [60]:
icgcann = sample[['project_code', 'submitted_sample_id', 'icgc_specimen_id', 'percentage_cellularity']]

In [62]:
for i, val in icgcann.iterrows():
    icgcann.loc[i, specimen.columns] = specimen.loc[val.icgc_specimen_id]

/home/jeremie/miniconda3/envs/gae/lib/python3.8/site-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


In [ ]:
icgcann['sequencer'] = "" # missing
icgcann['method'] = "bulk"
icgcann['reference']= "hg38"
icgcann['ends']="paired end"

## st jude

In [ ]:
pd.read_csv('') # 40,000 x 3500


## NCI 60

## tumor cell atlas

## other random datasets from SRA

## DUOS datasets

In [ ]:
#https://duos.broadinstitute.org/dataset_catalog

## hartwig

## Pancreas from Sri

## Neurosphere from Keith

## EBI

# QC

## compute correlation

- find unknown duplicates
- find problematic duplicates

In [204]:
import seaborn as sns

In [208]:
CCLE_expression[overlap] * TCGA_expression[overlap].T

,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,THR39_1369_S01,THR39_1370_S01,THR39_1371_S01,THR39_1372_S01,THR39_1373_S01,THR39_1374_S01,THR39_1375_S01,THR39_1376_S01,THR39_1377_S01,THR39_1378_S01
ACH-000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACH-000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACH-000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACH-000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACH-000005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000284873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000284892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000285077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000285135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
overlap = set(CCLE_expression.columns) & set(TCGA_expression.columns)
val = pd.concat([CCLE_expression[overlap],TCGA_expression[overlap]])
corr = val.corr()

In [ ]:
from s

## analyse  annotation files

- find mismatch annotations
- add missing annotations, need: 
    - sequencer, 
    - expression_type, 
    - media, 
    - origin, 
    - tissue type,
    - disease,
    - sub_disease,
    - cell type, 
    - group, 
    - sex, 
    - age, 
    - contamination
    - organism
- find good set of names for annotations

In [ ]:
ann[['tissue_type', 'disease_type', 'cell_type']]

In [215]:
ann = ann[ann.columns[ann.isna().sum()<800]]

In [216]:
AnnData(X=val, obs=ann).write('temp/celligner_multi.h5ad.gz', compression='gzip')

ValueError: Index of obs must match index of X.

In [ ]:
del ann, val
del CCLE_expression, TCGA_expression, gtex_v9

In [415]:
set(dataset.obs[dataset.obs.dataset.isin(['cmi-asc', 'cmi-mbc', 'hcmi-cmdc', 'organoid-pancreatic',])].lineage.astype(str) + "_" + dataset.obs[dataset.obs.dataset.isin(['cmi-asc', 'cmi-mbc', 'hcmi-cmdc', 'organoid-pancreatic'])].disease_type.astype(str) + "_" + dataset.obs[dataset.obs.dataset.isin(['cmi-asc', 'cmi-mbc', 'hcmi-cmdc', 'organoid-pancreatic'])].disease_subtype.astype(str))

/home/jeremie/miniconda3/envs/gae/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'bile_duct_cholangiocarcinoma_u',
 'bone_ewing_sarcoma_ewing_sarcoma',
 'brain_Secondary_malignant_neoplasm_of_brain_u',
 'brain_glioblastoma_glioblastoma',
 'brain_glioblastoma_u',
 'breast_ductal_carcinoma_in_situ,_nos_u',
 'breast_infiltrating_duct_and_lobular_carcinoma_u',
 'breast_infiltrating_duct_carcinoma,_nos_u',
 'breast_infiltrating_lobular_carcinoma,_nos_u',
 'colon_tubulovillous_adenoma,_nos_u',
 'gastric_adenocarcinoma,_intestinal_type_u',
 'kidney_nephroblastoma,_nos_u',
 'muscle_alveolar_rhabdomyosarcoma_rhabdomyosarcoma',
 'muscle_mixed_type_rhabdomyosarcoma_u',
 'soft_tissue_spindle_cell_sarcoma_u',
 'thyroid_medullary_carcinoma,_nos_u',
 'u_adenocarcinoma,_metastatic,_nos_u',
 'u_adenocarcinoma,_nos_u',
 'u_carcinoma,_nos_u',
 'u_malignant_melanoma,_nos_u',
 'u_metaplastic_carcinoma,_nos_u',
 'u_mucinous_adenocarcinoma_u',
 'u_serrated_adenoma_u',
 'u_u_u',
 'u_undifferentiated_sarcoma_sarcoma',
 'vascular_angiosarcoma_u'}

In [417]:
dataset.obs[dataset.obs.dataset.isin(['cmi-asc', 'cmi-mbc', 'hcmi-cmdc', 'organoid-pancreatic',])][(dataset.obs[dataset.obs.dataset.isin(['cmi-asc', 'cmi-mbc', 'hcmi-cmdc', 'organoid-pancreatic',])].lineage.astype(str) + "_" + dataset.obs[dataset.obs.dataset.isin(['cmi-asc', 'cmi-mbc', 'hcmi-cmdc', 'organoid-pancreatic'])].disease_type.astype(str) + "_" + dataset.obs[dataset.obs.dataset.isin(['cmi-asc', 'cmi-mbc', 'hcmi-cmdc', 'organoid-pancreatic'])].disease_subtype.astype(str)).isin(['u_adenocarcinoma,_metastatic,_nos_u',
 'u_adenocarcinoma,_nos_u',
 'u_carcinoma,_nos_u',
 'u_malignant_melanoma,_nos_u',
 'u_metaplastic_carcinoma,_nos_u',
 'u_mucinous_adenocarcinoma_u',
 'u_serrated_adenoma_u',
 'u_u_u',
 'u_undifferentiated_sarcoma_sarcoma'])].index

/home/jeremie/miniconda3/envs/gae/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['15e84d11-d9d3-4e2a-9487-9f1129d40255',
       'd73f4d76-8c91-4c90-877b-c757f7404cc1',
       'c312271b-a950-4a52-bba3-eabc4adc252a',
       '78ff3329-20bf-4bf6-b9f1-b346dc5e6e3a',
       'bba788ac-5786-42ee-a156-918b52ce5875',
       '368ea45f-8ee8-4e72-98fe-44d565cc304d',
       'b71babe9-1c46-4018-9410-1fc7e3c4f6c8',
       '6251bee4-967d-4532-9337-bc272fb76db2',
       '7721861d-e5ed-4cd5-81b7-5fc3bece5640',
       'd9bb36ee-51c7-4a3e-a72a-3337453d41cb',
       ...
       '99d3ab86-3688-4b05-8e86-f71b8dae82a5',
       '45b0fc1b-6965-4a94-90a3-83033c553edf',
       'd0a6f70f-eb87-4a36-8de8-22a4489544e7',
       '10e6418e-177b-401a-aef6-6027e93cc578',
       'a3e497c7-7716-4c65-b1a2-2ef5ecfe26f0',
       '0eb0f134-0889-41b6-b019-4584c6b1449b',
       'bd0d8e44-8b59-4800-8338-6bd13793045f',
       'b91f8e04-2960-4fd4-acb1-5ca39bf2c908',
       '80d0adea-d970-4f97-ac83-122084c727a7',
       '50b28db5-dea0-4473-a685-9cc8f0d01393'],
      dtype='object', length=208)

In [411]:
dataset.obs[dataset.obs.dataset=="cclf"][(dataset.obs[dataset.obs.dataset=="cclf"].lineage.astype(str) + "_" + dataset.obs[dataset.obs.dataset=="cclf"].disease_type.astype(str) + "_" + dataset.obs[dataset.obs.dataset=="cclf"].disease_subtype.astype(str)).isin(['u_adenocarcinoma_adenocarcinoma',
 'u_leg_mass_u',
 'u_rhabdoid_tumor_rhabdoid',
 'u_sarcoma_sarcoma',
 'u_squamous_cell_carcinoma_squamous_cell_carcinoma',
 'u_u_u',
 'u_undifferentiated_sarcoma_sarcoma',
 'thyroid_thyroid_ca_u',
 'thyroid_thyroid_cancer_u',])].index.tolist()

/home/jeremie/miniconda3/envs/gae/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['AB035-Tumor-SM-AS99X',
 'AB035-Tumor-SM-HPY1S',
 'BT856-Tumor-SM-AI7RN',
 'CCLF_Mix_RNASeq_June2016-AB002-Tumor-SM-5LB1D',
 'CCLF_Mix_RNASeq_June2016-AB002-Tumor-SM-5LB1O',
 'CCLF_Mix_RNASeq_June2016-JL30-Tumor-SM-78KL4',
 'CCLF_PEDS1037-Tumor-SM-HUQMS',
 'CCLF_PEDS1051-Tumor-SM-HUQNL',
 'HG002-Tumor-SM-HPY1X',
 'JL27-Tumor-SM-GXI9O',
 'JL36-Tumor-SM-7HEVF',
 'PEDS015-Tumor-SM-5LB4X',
 'RNAseq_05262017-CCLF_PEDS1010-Tumor-SM-E9I91',
 'RNAseq_05262017-CCLF_PEDS1010-Tumor-SM-E9I93',
 'RNAseq_05262017-CCLF_PEDS1010-Tumor-SM-E9I94',
 'RNAseq_05262017-PEDS014-Normal-SM-5LB5L',
 'RNAseq_05262017-PEDS014-Tumor-SM-5LB5P',
 'RNAseq_05262017-PEDS014-Tumor-SM-5LB5R',
 'RNAseq_05262017-PEDS014-Tumor-SM-79HUE',
 'RNAseq_05262017-RC2-Tumor-SM-E9AWO',
 'RNAseq_05262017-RH4-Tumor-SM-E9AWN',
 'RNAseq_05262017-RHJT-Tumor-SM-E9AWR',
 'RNAseq_05262017-RMS13-Tumor-SM-E9AWP',
 'RP-571_BT953_v2_RNA_OnPrem']

In [ ]:
dataset.obs[dataset.obs.dataset=="cclf"][(dataset.obs[dataset.obs.dataset=="cclf"].lineage.astype(str) + "_" + dataset.obs[dataset.obs.dataset=="cclf"].disease_type.astype(str) + "_" + dataset.obs[dataset.obs.dataset=="cclf"].disease_subtype.astype(str)).isin(['u_adenocarcinoma_adenocarcinoma',
 'u_leg_mass_u',
 'u_rhabdoid_tumor_rhabdoid',
 'u_sarcoma_sarcoma',
 'u_squamous_cell_carcinoma_squamous_cell_carcinoma',
 'u_u_u',
 'u_undifferentiated_sarcoma_sarcoma',
 'thyroid_thyroid_ca_u',
 'thyroid_thyroid_cancer_u',])].index.tolist()